# Jack the Reader: A Machine Reading Framework

## Prerequisites

**Note:** these commands need to be run in terminal from the root of Jack.
```
sh data/GloVe/download.sh
wget -O fastqa.zip https://www.dropbox.com/s/lftgh01zi60r9jv/fastqa.zip?dl=1
wget http://data.neuralnoise.com/jack-demo/esim.tar.gz
wget http://data.neuralnoise.com/jack-demo/bidaf.tar.gz
wget http://data.neuralnoise.com/jack2/nli/dam.tar.gz ###
unzip fastqa.zip
tar -xvzf bidaf.tar.gz
tar -xvzf esim.tar.gz
tar -xvzf dam.tar.gz
```

In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('..')    # change dir to Jack root
from jack import readers
from jack.core import QASetting
from jack.io.load import load_jack
from notebooks.prettyprint import QAPrettyPrint, print_nli

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Question Answering (QA)

Loading ready-to-use pretrained extractive QA models:

In [2]:
bidaf_reader = readers.reader_from_file("./bidaf")
fastqa_reader = readers.reader_from_file("./fastqa")

Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Restoring parameters from ./bidaf/model_module
INFO:tensorflow:Restoring parameters from ./fastqa/model_module


##### Example 1

In [69]:
paragraph = """It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. 
At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."""
question = "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?"

##### Example 2

In [31]:
paragraph = """Tom was visiting his family in the Rocky Mountains with his two little dogs. They had a welcome BBQ and ate all of the chocolates he brought. Tom has a friend whose family lives in Vermont."""
question = "Where do Tom's parents live?"
#question = "What did the family eat for desert?"
#question = "How many dogs does Tom have?"

##### Example 3

In [78]:
paragraph = "France won against Croatia 4 : 2 in yesterday's World Cup final. "
question = "What was the final score of the World Cup final match?"

##### Input Format: the `QASetting` data structure: 

In [79]:
qa_setting = QASetting(question=question, support=[paragraph])

##### Calling the Reader:

In [80]:
answers = bidaf_reader([qa_setting])
print(question)
QAPrettyPrint(paragraph, answers[0][0].span)

What was the final score of the World Cup final match?


##### Top-k Predictions with, Score:

In [18]:
fastqa_reader.model_module.set_topk(k=5)
answers = fastqa_reader([qa_setting])
for i, a in enumerate(answers[0]):
    print("Top %d Answer score: %.5f \t %s" % (i+1, a.score, a.text))

Top 1 Answer score: 0.94161 	 Saint Bernadette Soubirous
Top 2 Answer score: 0.94161 	 Saint Bernadette Soubirous
Top 3 Answer score: 0.05784 	 Bernadette Soubirous
Top 4 Answer score: 0.00042 	 Soubirous
Top 5 Answer score: 0.00009 	 Saint Bernadette


## Natural Language Inference (NLI)

Loading ready-to-use pretrained models:

In [19]:
esim_reader = readers.reader_from_file("./esim")

INFO:tensorflow:Restoring parameters from ./esim/model_module


##### Example 1

In [72]:
premise = "A wedding party is taking pictures."
hypothesis1 = "A group of people is celebrating."
hypothesis2 = "A rock band is giving a concert."

##### Example 2

In [40]:
premise = "France lost to Croatia 0:5 in yesterday's World Cup final."
hypothesis1 = "Croatia won against France in the World Cup finals."
hypothesis2 = "France won against Croatia in the World Cup finals"

The same `QASetting` input data structure is used:

In [73]:
snli_setting1 = QASetting(question=hypothesis1, support=[premise])
snli_setting2 = QASetting(question=hypothesis2, support=[premise])

We generate predictions by calling the reader with these inputs:

In [74]:
prediction = esim_reader([snli_setting1])
print_nli(premise, hypothesis1, prediction[0][0].text)

prediction = esim_reader([snli_setting2])
print_nli(premise, hypothesis2, prediction[0][0].text)

A wedding party is taking pictures.	--(entailment)-->	A group of people is celebrating.
A wedding party is taking pictures.	--(contradiction)-->	A rock band is giving a concert.


In the NLI case, the answer is a label among {_"entailment"_, _"neutral"_, _"contradiction"_}.

##### Prediction score:

In [23]:
print(prediction[0][0].score)

0.9988599
